In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

category_columns = [
    "Unlawful detention",
    "Human trafficking",
    "Enslavement",
    "Willful killing of civilians",
    "Mass execution",
    "Kidnapping",
    "Extrajudicial killing",
    "Forced disappearance",
    "Damage or destruction of civilian critical infrastructure",
    "Damage or destruction, looting, or theft of cultural heritage",
    "Military operations (battle, shelling)",
    "Gender-based or other conflict-related sexual violence",
    "Violent crackdowns on protesters/opponents/civil rights abuse",
    "Indiscriminate use of weapons",
    "Torture or indications of torture",
    "Persecution based on political, racial, ethnic, gender, or sexual orientation",
    "Movement of military, paramilitary, or other troops and equipment"
]

# 2) Custom Dataset class for articles
class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
train_df = pd.read_csv("train.csv")  
val_df   = pd.read_csv("val.csv")    
test_df  = pd.read_csv("test.csv")  

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text columns
train_encodings = tokenizer(
    list(train_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
val_encodings   = tokenizer(
    list(val_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
test_encodings  = tokenizer(
    list(test_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)

# Extract labels (multi-label targets in your category columns)
train_labels = train_df[category_columns].values
val_labels   = val_df[category_columns].values
test_labels  = test_df[category_columns].values

# Create Dataset objects
train_dataset = ArticleDataset(train_encodings, train_labels)
val_dataset   = ArticleDataset(val_encodings, val_labels)
test_dataset  = ArticleDataset(test_encodings, test_labels)


In [11]:
# Note: num_labels = number of category columns for multi-label classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(category_columns)
)

# Define compute_metrics for multi-label classification
def compute_metrics(p):
    # p.predictions are logits; p.label_ids are the ground truth
    preds = torch.sigmoid(torch.tensor(p.predictions))  # Convert logits to probabilities
    preds = (preds > 0.5).int().cpu().numpy() 
    labels = torch.tensor(p.label_ids).cpu().numpy()
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",      # Use F1 score for best model
    logging_dir='./logs'
)

# Initialize Trainer with training and validation sets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./bert-multiclass-model")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 21/1260 [00:14<10:04,  2.05it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn

{'eval_loss': 0.44863569736480713, 'eval_accuracy': 0.05263157894736842, 'eval_f1': 0.2951841065113377, 'eval_precision': 0.2799389778794813, 'eval_recall': 0.3565217391304348, 'eval_runtime': 0.7557, 'eval_samples_per_second': 75.429, 'eval_steps_per_second': 5.293, 'epoch': 1.0}


  3%|▎         | 42/1260 [00:30<10:07,  2.00it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor pr

{'eval_loss': 0.34153708815574646, 'eval_accuracy': 0.017543859649122806, 'eval_f1': 0.23878215930794291, 'eval_precision': 0.3019367588932806, 'eval_recall': 0.2956521739130435, 'eval_runtime': 0.7158, 'eval_samples_per_second': 79.626, 'eval_steps_per_second': 5.588, 'epoch': 2.0}


  5%|▌         | 63/1260 [00:49<09:47,  2.04it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor pr

...

{'eval_loss': 0.1764277070760727, 'eval_accuracy': 0.2982456140350877, 'eval_f1': 0.637878577108822, 'eval_precision': 0.7222701759488149, 'eval_recall': 0.6, 'eval_runtime': 0.775, 'eval_samples_per_second': 73.545, 'eval_steps_per_second': 5.161, 'epoch': 59.0}


100%|██████████| 1260/1260 [20:09<00:00,  1.90it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor 

{'eval_loss': 0.17636726796627045, 'eval_accuracy': 0.2982456140350877, 'eval_f1': 0.637878577108822, 'eval_precision': 0.7222701759488149, 'eval_recall': 0.6, 'eval_runtime': 0.9695, 'eval_samples_per_second': 58.792, 'eval_steps_per_second': 4.126, 'epoch': 60.0}


100%|██████████| 1260/1260 [20:29<00:00,  1.02it/s]


{'train_runtime': 1229.3785, 'train_samples_per_second': 15.764, 'train_steps_per_second': 1.025, 'train_loss': 0.1206161135718936, 'epoch': 60.0}


In [12]:
def evaluate_model(dataset, threshold=0.5):
    loader = DataLoader(dataset, batch_size=16)
    model.eval()  # Set model to eval mode
    
    preds_list = []
    labels_list = []
    
    with torch.no_grad():
        for batch in loader:
            inputs = {
                key: val.to(model.device) for key, val in batch.items() if key != 'labels'
            }
            outputs = model(**inputs)
            logits = outputs.logits
            # Convert logits to probabilities
            probs = torch.sigmoid(logits).cpu().numpy()
            # Apply threshold
            preds = (probs > threshold).astype(int)
            
            preds_list.extend(preds)
            labels_list.extend(batch['labels'].cpu().numpy())
    
    preds_array = np.array(preds_list)
    labels_array = np.array(labels_list)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_array, preds_array, average='weighted')
    acc = accuracy_score(labels_array, preds_array)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Run evaluation on the test dataset
test_results = evaluate_model(test_dataset)
print("Final Test Set Evaluation Results:", test_results)


Final Test Set Evaluation Results: {'accuracy': 0.3023255813953488, 'f1': 0.6284655723680114, 'precision': 0.7117797017797018, 'recall': 0.5777777777777777}


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to